In [40]:
library(tidyverse)
library(repr)
library(readxl)
library(ggplot2)

In [39]:
hungry <- read.csv("data/processed.hungarian.data", header = FALSE)

colnames(hungry) <- c("age", 'sex', 'chest_pain_type', 'resting_bps', 'cholestoral', 'fasting_blood_sugar', 'rest_ecg', 'max_heartrate', 'excercise_angina', 'depression', 'slope', 'major_vessels', 'thal', 'heart_disease')
hungry[hungry == "?"] <- NA


#hw1 coding team: visualize data (ggpair), wrangle, summarize, split training and testing data
# replace '?' to na

# hungry_clean <- hungry |>
#     select(-sex, -excercise_angina, -depression, -slope, -major_vessels, -thal, -rest_ecg, -fasting_blood_sugar) #|>
#    # replace_with_na(replace = list(cholestoral = "?")

hungry_pairs <- ggpair()

ERROR: Error in ggpair(): could not find function "ggpair"
